In [ ]:
!pip install openai  # OpenAI API for prompting GPT-4o

In [ ]:
import openpyxl
from openai import OpenAI
import os
import json
from tqdm import tqdm
import pandas as pd
import google.generativeai as genai

/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)


In [ ]:
#Import necessary libraries
from google.oauth2 import service_account
from googleapiclient.discovery import build

In [ ]:
# Import the API Key (insert key here)
GOOGLE_API_KEY=''
genai.configure(api_key=GOOGLE_API_KEY)

# Reading G-Drive Sheet

In [ ]:
!pip install --quiet gspread oauth2client google-auth
import gspread
from google.colab import auth
import pandas as pd
import google.auth

auth.authenticate_user()

# Authorize gspread client
# Use google.auth.default() to get credentials
credentials, project = google.auth.default()
gc = gspread.authorize(credentials)

def load_sheet(sheet_id, sheet_name="Sheet1"):
    # Load google sheet as dataframe
    worksheet = gc.open_by_key(sheet_id).worksheet(sheet_name)
    data = worksheet.get_all_values()
    df = pd.DataFrame(data[1:], columns=data[0])
    return df

path = "1WZlahRsgIBNi4l23SazRzBZ6K5xM_2j_eeTWzhEVsvE"
sheet = load_sheet(path,"roseWangResponses_mapped_20251112")  # There is only one sheet, so no ambiguity here.
sheet["GPT Rationale"] = ""

Get all the responses and scores

In [ ]:
INPUT_COLUMN = "response" # Specify the column from which input is read (A->1, B->2, etc.)
sheet = sheet[(sheet["title"]=="Affirming Student's Correct Attempt") & (sheet["prompt"].str.startswith(("3.","11.")))]
sheet = sheet.reset_index(drop=True)
row_count = len(sheet) # Get number of rows. If number of rows to read is already known (or sheet does not terminate where data terminates), place number here (+1) instead.inputs = [sheet.loc[i, INPUT_COLUMN]  for i in range(2, row_count)]  # Column number is set here. YOUR INPUT COLUMN NUMBER GOES HERE.
print(row_count)
inputs = [sheet.loc[i, INPUT_COLUMN] for i in range(row_count)]
inputs = [x for x in inputs if x is not None]
print(inputs)

303
['So that she knows that we care and that she is doing well.', "To show them that they're doing a good job and show them that they are appreciated.", 'To help them build confidence', 'Help them build confidence', 'It builds their confidence.', 'Builds their confidence', 'Because then they will feel satisfaction and feel happy', 'Because it will give the student confidence', 'It gives them motivation to get more questions right.', 'It gives them confidence.', 'This gives the student a robust strategy that they can use to solve the problem.', 'It gives her confidence to tackle similar problems in a similar manner.', 'it provides confidence and motivation', 'motivates and encourages', 'So that they are confident in their answer and encouraged to continue doing well.', 'To encourage and keep them motivated.', 'It gives them confidence', 'It will help them build self confidence', "It is important to affirm the student's answer because it lets them know they are following the right proce

# Prompt Setup

Scoring Prompt Input

In [ ]:
GEMINI_SYSTEM_PROMPT = """
You are a tutor evaluator. Please score the following tutor response to a tutor training scenario involving a middle school student:
-if the tutor's response demonstrates an understanding of how affirmation provides positive reinforcement and benefits the learning process, score with a 1. Examples of responses scoring a 1 are: “So that she knows that we care and that she is doing well.”; “it provides confidence and motivation.”; “It provides positive reinforcement. This helps the student stay motivated and enjoy the math”.
-if the tutor response does NOT demonstrate an understanding of how affirmation provides positive reinforcement and benefits the learning process, score with a 0. Sample responses scoring a 0 include: “Positive feedback feels good!”; “So they get happy when they get it right not just disappointed when they get it wrong.”.

Once given a response by the user, please return a JSON string following the format, {"Rationale": "your reasoning here", "Score":0/1}
"""

Helper function for response parsing

In [ ]:
def extract_response(response_obj, json=False):
  role = response_obj.choices[0].message.role
  content = response_obj.choices[0].message.content
  if json:
    return {"role": role, "content": content}
  else:
    return (role, content)

## OpenAI API Call

In [ ]:
# Iterate over all responses
MAX_TOKENS = 1000
TEMPERATURE = 0
RUN_UP_TO = 303  # Sets a maximum index for responses to run. Useful to specify how many responses we want to run on (partial execution). Set to -1 to run them all.
SCORE_COLUMN = "response"  # Change column numbers here to  modify where output is written

MODEL = 'gemini-2.5-pro'
model = genai.GenerativeModel(MODEL, system_instruction=GEMINI_SYSTEM_PROMPT)


if RUN_UP_TO >=  0:  # If an upper bound is set
  inputs_upto = inputs[:RUN_UP_TO]
else:
  inputs_upto = inputs  # Take the whole set of responses
tempScoreList = []
tempDirectionList = []
tempRationaleList = []

for index, inpt in tqdm(enumerate(inputs_upto), total=len(inputs_upto)):
  generation_prompt = "Tutor Response: " + inpt + "\n\n. Your JSON: " # Gemini Change
  generation_config = genai.GenerationConfig(temperature=TEMPERATURE)
  # Use client.chat.completions.create instead of client.generate_text
  gemini_out = model.generate_content(generation_prompt, generation_config=generation_config) # TODO: Potential break point
  print(gemini_out)
  # Extract the content from the response
  content = gemini_out.text.lstrip("```json")[:-4]
  print(content)
  # We now need to parse the JSON into rational and score

  try:
    content_json = json.loads(content)  # Run response through JSON
    score = str(content_json["Score"])  # Cast to string to avoid type inequality
    rationale = str(content_json["Rationale"])  # Fetch the rationale
    sheet.at[index,"Gemini Score"] = score  # Now write both into the dataframe
    sheet.at[index,"Gemini Rationale"] = rationale
    #score_cell.value = score
    #rationale_cell.value = rationale
    tempScoreList.append(score)
    tempDirectionList.append(generation_prompt)
    tempRationaleList.append(rationale)

  except:
    print("error!")
  #   score_cell.value = "---"
  #   rationale_cell.value = "---"  # Failsafe
  #   if TWO_STAGE:
  #     refined_feedback_cell.value = "---"
print(tempScoreList)
print(tempDirectionList)
print(tempRationaleList)

  0%|          | 1/303 [00:08<43:04,  8.56s/it]

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "```json\n{\n\"Rationale\": \"This response correctly identifies two key benefits of affirmation in the learning process: building a supportive tutor-student relationship ('knows that we care') and providing positive reinforcement about the student's performance ('knows that she is doing well'). This demonstrates an understanding of how affirmation builds confidence and motivates the student, directly aligning with the criteria for a score of 1.\",\n\"Score\":1\n}\n```"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 225,
        "candidates_token_count": 91,
        "total_token_count": 1158
      },
     

  1%|          | 2/303 [00:18<46:02,  9.18s/it]

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "```json\n{\n\"Rationale\": \"The tutor's response explains that affirmation is used to 'show them that they're doing a good job.' This directly links the praise to the student's successful work, which is a form of positive reinforcement. This acknowledgment helps build confidence and motivation, thereby benefiting the learning process.\",\n\"Score\":1\n}\n```"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 230,
        "candidates_token_count": 76,
        "total_token_count": 1141
      },
      "model_version": "gemini-2.5-pro"
    }),
)

{
"Rationale": "The tutor's response explains that affirmation is

  1%|          | 3/303 [00:26<43:24,  8.68s/it]

...

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "```json\n{\n\"Rationale\": \"The tutor's response correctly identifies that affirmation helps students become more confident and motivated. This demonstrates a clear understanding of how positive reinforcement benefits the learning process by building a student's self-efficacy and drive to continue learning, which aligns with the criteria for a score of 1.\",\n\"Score\":1\n}\n```"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 221,
        "candidates_token_count": 75,
        "total_token_count": 1099
      },
      "model_version": "gemini-2.5-pro"
    }),
)

{
"Rationale": "The tutor's response correct

100%|█████████▉| 302/303 [44:01<00:08,  8.34s/it]

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "```json\n{\n\"Rationale\": \"The tutor's response correctly identifies that affirmation serves as a tool for motivation ('keeps them motivated') and encourages persistence ('encourages them to keep trying'). This demonstrates a clear understanding of how positive reinforcement benefits the learning process by fostering resilience and engagement.\",\n\"Score\":1\n}\n```"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 222,
        "candidates_token_count": 68,
        "total_token_count": 1103
      },
      "model_version": "gemini-2.5-pro"
    }),
)

{
"Rationale": "The tutor's response correctly identifi

100%|██████████| 303/303 [44:09<00:00,  8.74s/it]

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "```json\n{\n\"Rationale\": \"The tutor's response correctly identifies that affirmation boosts a student's confidence. This is a key component of positive reinforcement and directly benefits the learning process by making a student more willing to engage and persevere. The response aligns with the provided example for a score of 1, 'it provides confidence and motivation.'\",\n\"Score\":1\n}\n```"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "index": 0
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 216,
        "candidates_token_count": 81,
        "total_token_count": 1190
      },
      "model_version": "gemini-2.5-pro"
    }),
)

{
"Rationale": "The tutor's 

In [ ]:
for i in range(len(inputs_upto)):
  print(inputs_upto[i])
  print(tempScoreList[i])
  print(tempRationaleList[i])
  print()

So that she knows that we care and that she is doing well.
1
This response correctly identifies two key benefits of affirmation in the learning process: building a supportive tutor-student relationship ('knows that we care') and providing positive reinforcement about the student's performance ('knows that she is doing well'). This demonstrates an understanding of how affirmation builds confidence and motivates the student, directly aligning with the criteria for a score of 1.

To show them that they're doing a good job and show them that they are appreciated.
1
The tutor's response explains that affirmation is used to 'show them that they're doing a good job.' This directly links the praise to the student's successful work, which is a form of positive reinforcement. This acknowledgment helps build confidence and motivation, thereby benefiting the learning process.

To help them build confidence
1
The tutor's response correctly identifies that affirmation helps to build confidence. Buil

In [ ]:
print(list(sheet["GPT Score"]))

['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '',

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Save G-Drive Sheet

In [ ]:
sheet.to_csv('/content/drive/Shareddrives/PLUS/Research/Clara Stuff/Affirming_Students_Correct_Attempt_Explain_Gemini_Eval.csv', index=False)